<a href="https://colab.research.google.com/github/Tirthankar4/Kaggle/blob/main/July_2024_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e7:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F73291%2F8930475%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240729%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240729T110509Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da0510c2357178b7eb8fa8fc2f1d6c35d6df4b7b28be5cd184aad636ab17f0df0f118ca7deeadd4effbf1d6bfa07c1db29d75d87f99244d1f4fb17167960bff5852f7ff6aee3769e9f06ab7b8fdc3339a9df14ac28ccbd164a8f269e50ae02a915f546c0662a02e13215c8b6748f4b9eb592f75fbd0009c3ff94a25a6378a03a73ec544e61cb85ad2c17d714ab9eac733d08b0ed2c1084b83f92e25ae16ae30908b690a648001af6ed1fa8731d1c8700f9fc56e021c94f344634f513c5e20ab7a6da1556e32e8cc17efaeccf38c1dfededba09526efd4032954a6a4e07b0648813d8337862152fa04e3d6dd95bda6935d69e396ece38c14302cd55fbc32dd2050'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier, Pool
#import optuna
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
import warnings
import logging

warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.CRITICAL)

In [ ]:
train = pd.read_csv('../input/playground-series-s4e7/train.csv', index_col = 'id')
test = pd.read_csv('../input/playground-series-s4e7/test.csv', index_col = 'id')

In [ ]:
def converting_datatypes(df):
    df = df.copy()
    try:
        df['Gender'] = df['Gender'].astype('category')
        df['Vehicle_Age'] = df['Vehicle_Age'].astype('category')
        df['Vehicle_Damage'] = df['Vehicle_Damage'].astype('category')
        df['Age'] = df['Age'].astype('int8')
        df['Driving_License'] = df['Driving_License'].astype('int8')
        df['Region_Code'] = df['Region_Code'].astype('int8')
        df['Previously_Insured'] = df['Previously_Insured'].astype('int8')
        df['Annual_Premium'] = df['Annual_Premium'].astype('int32')
        df['Policy_Sales_Channel'] = df['Policy_Sales_Channel'].astype('int16')
        df['Vintage'] = df['Vintage'].astype('int16')
        df['Response'] = df['Response'].astype('int8')
    except KeyError as e:
        print(f"Error: {e} not found in DataFrame")
    except Exception as e:
        print(f"An error occurred: {e}")
    return df

train = converting_datatypes(train)
test = converting_datatypes(test)

In [ ]:
y = train.pop('Response')

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size = 0.2, random_state = 69)

In [ ]:
gender = LabelEncoder()

X_train['Gender'] = gender.fit_transform(X_train['Gender'])
X_valid['Gender'] = gender.transform(X_valid['Gender'])
test['Gender'] = gender.transform(test['Gender'])

damaged = LabelEncoder()

X_train['Vehicle_Damage'] = damaged.fit_transform(X_train['Vehicle_Damage'])
X_valid['Vehicle_Damage'] = damaged.transform(X_valid['Vehicle_Damage'])
test['Vehicle_Damage'] = damaged.transform(test['Vehicle_Damage'])

age = LabelEncoder()

X_train['Vehicle_Age'] = age.fit_transform(X_train['Vehicle_Age'])
X_valid['Vehicle_Age'] = age.transform(X_valid['Vehicle_Age'])
test['Vehicle_Age'] = age.transform(test['Vehicle_Age'])

X_train["Insured_Damage"] = pd.factorize(X_train['Previously_Insured'].astype(str) + X_train['Vehicle_Damage'].astype(str))[0]
X_valid["Insured_Damage"] = pd.factorize(X_valid['Previously_Insured'].astype(str) + X_valid['Vehicle_Damage'].astype(str))[0]
test["Insured_Damage"] = pd.factorize(test['Previously_Insured'].astype(str) + test['Vehicle_Damage'].astype(str))[0]

X_train["Insured_Premium"] = pd.factorize(X_train['Previously_Insured'].astype(str) + X_train['Annual_Premium'].astype(str))[0]
X_valid["Insured_Premium"] = pd.factorize(X_valid['Previously_Insured'].astype(str) + X_valid['Annual_Premium'].astype(str))[0]
test["Insured_Premium"] = pd.factorize(test['Previously_Insured'].astype(str) + test['Annual_Premium'].astype(str))[0]

X_train["Insured_Age"] = pd.factorize(X_train['Previously_Insured'].astype(str) + X_train['Vehicle_Age'].astype(str))[0]
X_valid["Insured_Age"] = pd.factorize(X_valid['Previously_Insured'].astype(str) + X_valid['Vehicle_Age'].astype(str))[0]
test["Insured_Age"] = pd.factorize(test['Previously_Insured'].astype(str) + test['Vehicle_Age'].astype(str))[0]

In [ ]:
xgb_skf = StratifiedKFold(n_splits=10, shuffle=True, random_state = 69)

xgb_valid_auc = []
xgb_models = []

for train_index, valid_index in xgb_skf.split(X_train, y_train):
    X_train_fold, X_valid_fold = X_train.iloc[train_index], X_train.iloc[valid_index]
    y_train_fold, y_valid_fold = y_train.iloc[train_index], y_train.iloc[valid_index]

    def objective(trial):

        xgb_param = {
        'eval_metric': 'auc',
        'reg_lambda': trial.suggest_float('lambda', 20.0, 69.0),
        'max_depth': trial.suggest_int('max_depth', 10, 20),
        'eta': trial.suggest_float('eta', 0.02, 0.5),
        'subsample': trial.suggest_float('subsample', 0.69, 0.96),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.49, 0.96),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.49, 0.96),
        'tree_method': 'hist',
        'scale_pos_weight': 7.33,
        'n_estimators': 3996,
        'early_stopping_rounds': 196,
        'random_state': 69,
        'device': 'cuda',
        'max_bin': 262146
    }

        model = XGBClassifier(**xgb_param)
        model.fit(X_train_fold, y_train_fold, eval_set = [(X_valid_fold, y_valid_fold)], verbose = False)
        fold_preds = model.predict_proba(X_valid_fold)[:, 1]
        score = roc_auc_score(y_valid_fold, fold_preds)
        return score

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=69)

    best_params = study.best_params
    best_score = study.best_value

    xgb_valid_auc.append(best_score)
    xgb_models.append(best_params)

best_xgb_index = xgb_valid_auc.index(max(xgb_valid_auc))
best_xgb_params = xgb_models[best_xgb_index]

In [ ]:
xgb_model = XGBClassifier(**best_xgb_params)
xgb_model.fit(X_train, y_train, verbose=696)

xgb_preds = xgb_model.predict_proba(test)[:, 1]

In [ ]:
for col in X_train.columns:

    X_train[col] = X_train[col].astype('category')
    X_valid[col] = X_valid[col].astype('category')
    test[col] = test[col].astype('category')

In [ ]:
cat_skf = StratifiedKFold(n_splits=10, shuffle=True, random_state = 69)

cat_valid_auc = []
cat_models = []

for train_index, valid_index in cat_skf.split(X_train, y_train):
    X_train_fold, X_valid_fold = X_train.iloc[train_index], X_train.iloc[valid_index]
    y_train_fold, y_valid_fold = y_train.iloc[train_index], y_train.iloc[valid_index]

    X_train_pool = Pool(X_train_fold, y_train_fold, cat_features = X_train_fold.columns.values)
    X_valid_pool = Pool(X_valid_fold, y_valid_fold, cat_features = X_valid_fold.columns.values)

    def objective(trial):

        cat_param = {
        'eval_metric': 'AUC',
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.1, 1.0),
        'max_depth': trial.suggest_int('max_depth', 10, 16),
        'eta': trial.suggest_float('eta', 0.02, 0.5),
        'n_estimators': 3996,
        'early_stopping_rounds':96,
        'scale_pos_weight': 7.33,
        'task_type': 'GPU',
        'fold_permutation_block': 64,
        'random_state': 69,
    }

        model = CatBoostClassifier(**cat_param)
        model.fit(X_train_pool, eval_set = X_valid_pool, verbose = False)
        fold_preds = model.predict_proba(X_valid_fold)[:, 1]
        score = roc_auc_score(y_valid_fold, fold_preds)
        return score

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=69)

    best_params = study.best_params
    best_score = study.best_value

    cat_valid_auc.append(best_score)
    cat_models.append(best_params)

best_cat_index = cat_valid_auc.index(max(cat_valid_auc))
best_cat_params = cat_models[best_cat_index]

In [ ]:
cat_model = CatBoostClassifier(**best_cat_params)
X_train_pool = Pool(X_train, y_train, cat_features=X_train.columns.values)
cat_model.fit(X_train_pool, verbose = 696)

cat_preds = cat_model.predict_proba(test)[:, 1]

In [ ]:
xgb_output = pd.DataFrame({'id': test.index,'Response': xgb_preds})

xgb_output.to_csv('xgb.csv', index=False)

In [ ]:
cat_output = pd.DataFrame({'id': test.index,'Response': cat_preds})

cat_output.to_csv('cat.csv', index=False)